# LIBRERIAS

In [ ]:
import pandas as pd
import random

# CODIGO CON ITERACIONES

In [ ]:
def organizar_entregas(C):
    C = C.sort_values(by=' Fecha Limite').reset_index(drop=True)
    return C

def seleccion_aleatoria(C):
    C = organizar_entregas(C)
    pesos = [1 / (i + 1) for i in range(len(C))]
    Orden_utilizado = []
    indices_disponibles = list(range(len(C)))
    C_ordenado = pd.DataFrame(columns=C.columns)

    while indices_disponibles:
        pesos_disponibles = [pesos[i] for i in indices_disponibles]
        total_pesos = sum(pesos_disponibles)
        probabilidades = [p / total_pesos for p in pesos_disponibles]
        numero_seleccionado = random.choices(indices_disponibles, weights=probabilidades, k=1)[0]
        Orden_utilizado.append(numero_seleccionado)
        C_ordenado = pd.concat([C_ordenado, C.iloc[[numero_seleccionado]]])
        indices_disponibles.remove(numero_seleccionado)
    return C_ordenado

def preprocesar_rates(df2):
    rates = {}
    for _, row in df2.iterrows():
        rates[(row['IdMaquina'], row['IdProducto'])] = row['Rate']
    return rates



def acomodo(df, df2):
    maquinas_fase1 = {0: 0, 1: 0, 2: 0}
    hornos = {3: 0, 4: 0}
    maquinas_fase3 = {5: 0, 6: 0, 7: 0}

    last_product_fase1 = {0: None, 1: None, 2: None}
    last_product_fase2 = {3: None, 4: None}
    last_product_fase3 = {5: None, 6: None, 7: None}

    hora_inicio = 0
    hora_limite = 16
    dia_actual = 0
    pedidos_tardios = 0

    set_up_maquina_misma = 0.25
    set_up_maquina_distinta = 0.5
    set_up_horno_misma = 0.75
    set_up_horno_distinta = 1

    rates = preprocesar_rates(df2)
    rutas_pedidos = []

    dias_semana = ["Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado", "Domingo"]

    for i, pedido in df.iterrows():
        id_producto = pedido[' IdProducto']
        cantidad = pedido['Cantidad']
        fecha_limite = pedido[' Fecha Limite']

        maquina_fase1 = min(maquinas_fase1, key=maquinas_fase1.get)
        setup_time_fase1 = set_up_maquina_distinta if last_product_fase1[maquina_fase1] not in (None, id_producto) else set_up_maquina_misma
        tiempo_fase1 = cantidad / rates.get((maquina_fase1, id_producto), 0) + setup_time_fase1
        inicio_fase1 = maquinas_fase1[maquina_fase1]
        fin_fase1 = inicio_fase1 + tiempo_fase1
        maquinas_fase1[maquina_fase1] += tiempo_fase1
        last_product_fase1[maquina_fase1] = id_producto

        horno = min(hornos, key=hornos.get)
        setup_time_fase2 = set_up_horno_distinta if last_product_fase2[horno] not in (None, id_producto) else set_up_horno_misma
        inicio_fase2 = max(hornos[horno], fin_fase1)
        tiempo_fase2 = cantidad / rates.get((horno, id_producto), 0) + setup_time_fase2
        fin_fase2 = inicio_fase2 + tiempo_fase2
        hornos[horno] = fin_fase2
        last_product_fase2[horno] = id_producto

        maquina_fase3 = min(maquinas_fase3, key=maquinas_fase3.get)
        setup_time_fase3 = set_up_maquina_distinta if last_product_fase3[maquina_fase3] not in (None, id_producto) else set_up_maquina_misma
        inicio_fase3 = max(maquinas_fase3[maquina_fase3], fin_fase2)
        tiempo_fase3 = cantidad / rates.get((maquina_fase3, id_producto), 0) + setup_time_fase3
        fin_fase3 = inicio_fase3 + tiempo_fase3
        maquinas_fase3[maquina_fase3] = fin_fase3
        last_product_fase3[maquina_fase3] = id_producto

        hora_entrega = fin_fase3

        if hora_entrega > hora_limite:
            dia_actual += 1
            maquinas_fase1 = {k: 0 for k in maquinas_fase1}
            hornos = {k: 0 for k in hornos}
            maquinas_fase3 = {k: 0 for k in maquinas_fase3}
            last_product_fase1 = {k: None for k in last_product_fase1}
            last_product_fase2 = {k: None for k in last_product_fase2}
            last_product_fase3 = {k: None for k in last_product_fase3}
            hora_inicio = 0
            hora_limite = 16
            hora_entrega = fin_fase3
            maquina_fase1 = min(maquinas_fase1, key=maquinas_fase1.get)
            setup_time_fase1 = set_up_maquina_distinta if last_product_fase1[maquina_fase1] not in (None, id_producto) else set_up_maquina_misma
            tiempo_fase1 = cantidad / rates.get((maquina_fase1, id_producto), 0) + setup_time_fase1
            inicio_fase1 = maquinas_fase1[maquina_fase1]
            fin_fase1 = inicio_fase1 + tiempo_fase1
            maquinas_fase1[maquina_fase1] += tiempo_fase1
            last_product_fase1[maquina_fase1] = id_producto
            horno = min(hornos, key=hornos.get)
            setup_time_fase2 = set_up_horno_distinta if last_product_fase2[horno] not in (None, id_producto) else set_up_horno_misma
            inicio_fase2 = max(hornos[horno], fin_fase1)
            tiempo_fase2 = cantidad / rates.get((horno, id_producto), 0) + setup_time_fase2
            fin_fase2 = inicio_fase2 + tiempo_fase2
            hornos[horno] = fin_fase2
            last_product_fase2[horno] = id_producto
            maquina_fase3 = min(maquinas_fase3, key=maquinas_fase3.get)
            setup_time_fase3 = set_up_maquina_distinta if last_product_fase3[maquina_fase3] not in (None, id_producto) else set_up_maquina_misma
            inicio_fase3 = max(maquinas_fase3[maquina_fase3], fin_fase2)
            tiempo_fase3 = cantidad / rates.get((maquina_fase3, id_producto), 0) + setup_time_fase3
            fin_fase3 = inicio_fase3 + tiempo_fase3
            maquinas_fase3[maquina_fase3] = fin_fase3
            last_product_fase3[maquina_fase3] = id_producto


        estatus = "a tiempo" if dia_actual <= fecha_limite else "entrega tardia"
        if estatus == "entrega tardia":
            pedidos_tardios += 1

        dia_inicio_nombre = dias_semana[dia_actual % 7]
        hora_inicio_fase1_ajustada = inicio_fase1 + 6
        hora_fin_fase1_ajustada = fin_fase1 + 6
        hora_inicio_fase2_ajustada = inicio_fase2 + 6
        hora_fin_fase2_ajustada = fin_fase2 + 6
        hora_inicio_fase3_ajustada = inicio_fase3 + 6
        hora_fin_fase3_ajustada = fin_fase3 + 6

        def formato_hora(hora):
            return f"{int(hora):02d}:{int((hora % 1) * 60):02d}"

        ruta = {
            "Numero de pedido": pedido['IdExperimento'],
            "Dia de inicio": dia_inicio_nombre,
            "Maquina Fase 1": f"Maquina {maquina_fase1}",
            "Fase 1 Hora de Inicio": formato_hora(hora_inicio_fase1_ajustada),
            "Fase 1 Hora de Fin": formato_hora(hora_fin_fase1_ajustada),
            "Horno Fase 2": f"Horno {horno}",
            "Fase 2 Hora de Inicio": formato_hora(hora_inicio_fase2_ajustada),
            "Fase 2 Hora de Fin": formato_hora(hora_fin_fase2_ajustada),
            "Maquina Fase 3": f"Maquina {maquina_fase3}",
            "Fase 3 Hora de Inicio": formato_hora(hora_inicio_fase3_ajustada),
            "Fase 3 Hora de Fin": formato_hora(hora_fin_fase3_ajustada),
            "Dia de salida": dia_inicio_nombre,
            "Hora de salida": formato_hora(hora_fin_fase3_ajustada),
            "Estatus": estatus
        }
        rutas_pedidos.append(ruta)

    return pedidos_tardios, rutas_pedidos


def iteraciones(df, df2, max_iteraciones=10000):
    mejor_solucion = None
    menor_tardios = float('inf')
    for _ in range(max_iteraciones):
        pedidos_aleatorios = seleccion_aleatoria(df)
        pedidos_tardios, rutas = acomodo(pedidos_aleatorios, df2)
        if pedidos_tardios < menor_tardios:
            menor_tardios = pedidos_tardios
            mejor_solucion = rutas
        if pedidos_tardios == 0:
            break
    print(f"\nMejor solución encontrada con {menor_tardios} pedidos tardíos")
    if mejor_solucion:
        rutas_df = pd.DataFrame(mejor_solucion)
        print(rutas_df)
        rutas_df.to_csv('rutas.csv', index=False)


# PRUEBA CON POCOS PEDIDOS

MAXIMO 10 POR PEDIDO

14 PEDIDOS

14 PRODUCTOS

RATE MAQUINAS (10-20)

RATE HORNOS (5-9)

In [ ]:
df = pd.read_csv('PEDIDOS1.csv')
df.drop("IdPedido",axis = 1,inplace=True)
print(df)

    IdExperimento  Cantidad   Fecha Limite   IdProducto
0               0         2              2            0
1               1         8              4           10
2               2         5              6            0
3               3         9              6            6
4               4         7              4            7
5               5         3              3           10
6               6         3              5            7
7               7         2              1            4
8               8         9              2            3
9               9         2              4           12
10             10         4              3            1
11             11         6              4            4
12             12         9              2            5
13             13         2              4            9


In [ ]:
df2 = pd.read_csv('RATES1.csv')
print(df2)

     IdMaquina  IdProducto  Rate
0            0           0    19
1            0           1    10
2            0           2    15
3            0           3    14
4            0           4    12
..         ...         ...   ...
107          7           9    12
108          7          10    17
109          7          11    12
110          7          12    13
111          7          13    15

[112 rows x 3 columns]


In [ ]:
pedidos_aleatorios = seleccion_aleatoria(df)
iteraciones(pedidos_aleatorios, df2, max_iteraciones=100)


Mejor solución encontrada con 0 pedidos tardíos
    Numero de pedido Dia de inicio Maquina Fase 1 Fase 1 Hora de Inicio  \
0                  7         Lunes      Maquina 0                 06:00   
1                 11         Lunes      Maquina 1                 06:00   
2                  0         Lunes      Maquina 2                 06:00   
3                 10         Lunes      Maquina 0                 06:25   
4                  3         Lunes      Maquina 2                 06:27   
5                  6         Lunes      Maquina 1                 06:50   
6                 12         Lunes      Maquina 0                 07:18   
7                  9         Lunes      Maquina 1                 07:34   
8                  8         Lunes      Maquina 2                 07:38   
9                  4         Lunes      Maquina 1                 08:11   
10                 1         Lunes      Maquina 0                 08:18   
11                 5         Lunes      Maquina 2  

# PRUEBA CON PEDIDOS MODERADOS

MAXIMO 15 POR PEDIDO

35 PEDIDOS

14 PRODUCTOS

RATE MAQUINAS (10-20)

RATE HORNOS (4-8)

In [ ]:
df3 = pd.read_csv('PEDIDOS2.csv')
df3.drop("IdPedido",axis = 1,inplace=True)
print(df3)

    IdExperimento  Cantidad   Fecha Limite   IdProducto
0               0         8              2            6
1               1        13              5            9
2               2         9              4            9
3               3         1              0           11
4               4         9              6            0
5               5         4              5            1
6               6        11              2           13
7               7        12              5           13
8               8         3              4            5
9               9         6              4            2
10             10        11              1            2
11             11        12              0            1
12             12        13              2            0
13             13        13              6            9
14             14         2              6            9
15             15         8              5            5
16             16        12              3      

In [ ]:
df4 = pd.read_csv('RATES2.csv')
print(df4)

     IdMaquina  IdProducto  Rate
0            0           0    12
1            0           1    10
2            0           2    14
3            0           3    19
4            0           4    10
..         ...         ...   ...
107          7           9    13
108          7          10    18
109          7          11    13
110          7          12    11
111          7          13    18

[112 rows x 3 columns]


In [ ]:
pedidos_aleatorios2 = seleccion_aleatoria(df3)
iteraciones(pedidos_aleatorios2, df4, max_iteraciones=100)


Mejor solución encontrada con 0 pedidos tardíos
    Numero de pedido Dia de inicio Maquina Fase 1 Fase 1 Hora de Inicio  \
0                 11         Lunes      Maquina 0                 06:00   
1                  3         Lunes      Maquina 1                 06:00   
2                 10         Lunes      Maquina 2                 06:00   
3                 12         Lunes      Maquina 1                 06:18   
4                 19         Lunes      Maquina 2                 07:15   
5                  6         Lunes      Maquina 0                 07:27   
6                 20         Lunes      Maquina 1                 07:29   
7                 30         Lunes      Maquina 2                 07:55   
8                  1         Lunes      Maquina 1                 08:42   
9                 29        Martes      Maquina 0                 06:00   
10                31        Martes      Maquina 1                 06:00   
11                21        Martes      Maquina 2  

# PRUEBA CON MUCHOS PEDIDOS

MAXIMO 17 POR PEDIDO

65 PEDIDOS

14 PRODUCTOS

RATE MAQUINAS (9-18)

RATE HORNOS (4-8)

In [ ]:
df5 = pd.read_csv('PEDIDOS3.csv')
df5.drop("IdPedido",axis = 1,inplace=True)
print(df5)

    IdExperimento  Cantidad   Fecha Limite   IdProducto
0               0        13              5            3
1               1        14              6            0
2               2         5              3            4
3               3        13              0           12
4               4         7              3            9
..            ...       ...            ...          ...
60             60        16              4            5
61             61         4              0           12
62             62         6              1            7
63             63         3              6            5
64             64         8              1            8

[65 rows x 4 columns]


In [ ]:
df6 = pd.read_csv('RATES3.csv')
print(df6)

     IdMaquina  IdProducto  Rate
0            0           0    14
1            0           1    14
2            0           2    13
3            0           3    14
4            0           4    15
..         ...         ...   ...
107          7           9    14
108          7          10    17
109          7          11    10
110          7          12    14
111          7          13    14

[112 rows x 3 columns]


In [ ]:
pedidos_aleatorios3 = seleccion_aleatoria(df5)
iteraciones(pedidos_aleatorios3, df6, max_iteraciones=10000)


Mejor solución encontrada con 7 pedidos tardíos
    Numero de pedido Dia de inicio Maquina Fase 1 Fase 1 Hora de Inicio  \
0                 51         Lunes      Maquina 0                 06:00   
1                 52         Lunes      Maquina 1                 06:00   
2                 26         Lunes      Maquina 2                 06:00   
3                 17         Lunes      Maquina 0                 06:12   
4                  9         Lunes      Maquina 2                 06:14   
..               ...           ...            ...                   ...   
60                38        Sabado      Maquina 2                 06:12   
61                54        Sabado      Maquina 0                 06:42   
62                20        Sabado      Maquina 1                 07:13   
63                39        Sabado      Maquina 0                 07:31   
64                43        Sabado      Maquina 2                 07:46   

   Fase 1 Hora de Fin Horno Fase 2 Fase 2 Hora de 

# CASO EXTREMO

MAXIMO 18 POR PEDIDO

100 PEDIDOS

14 PRODUCTOS

RATE MAQUINAS (8-17)

RATE HORNOS (3-7)

In [ ]:
df7 = pd.read_csv('PEDIDOS4.csv')
df7.drop("IdPedido",axis = 1,inplace=True)
print(df7)

    IdExperimento  Cantidad   Fecha Limite   IdProducto
0               0         1              6            5
1               1        17              3            4
2               2        17              4            3
3               3        16              0            1
4               4        10              0            8
..            ...       ...            ...          ...
95             95         1              3            5
96             96        11              1            6
97             97        13              2           12
98             98         1              2           13
99             99         3              6            9

[100 rows x 4 columns]


In [ ]:
df8 = pd.read_csv('RATES4.csv')
print(df8)

     IdMaquina  IdProducto  Rate
0            0           0    11
1            0           1    11
2            0           2    14
3            0           3    15
4            0           4    11
..         ...         ...   ...
107          7           9    14
108          7          10    13
109          7          11    11
110          7          12    14
111          7          13    14

[112 rows x 3 columns]


In [ ]:
pedidos_aleatorios4 = seleccion_aleatoria(df7)
iteraciones(pedidos_aleatorios4, df8, max_iteraciones=15000)


Mejor solución encontrada con 58 pedidos tardíos
    Numero de pedido Dia de inicio Maquina Fase 1 Fase 1 Hora de Inicio  \
0                 52         Lunes      Maquina 0                 06:00   
1                 82         Lunes      Maquina 1                 06:00   
2                 36         Lunes      Maquina 2                 06:00   
3                 90         Lunes      Maquina 1                 06:17   
4                 22         Lunes      Maquina 2                 06:19   
..               ...           ...            ...                   ...   
95                99       Viernes      Maquina 2                 07:15   
96                88       Viernes      Maquina 0                 07:32   
97                80       Viernes      Maquina 2                 07:58   
98                18        Sabado      Maquina 0                 06:00   
99                20        Sabado      Maquina 1                 06:00   

   Fase 1 Hora de Fin Horno Fase 2 Fase 2 Hora de